In [1]:
import re

def parse_facts(file_path):
    facts = {}
    target = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("target:"):
                target_line = line[len("target:"):].strip()
                key, op, val = parse_condition(target_line)
                target = (key, val)  # Only store key and expected value
                continue
            if '=' in line:
                key, val = line.split('=')
                try:
                    facts[key.strip()] = int(val.strip())
                except:
                    facts[key.strip()] = val.strip()
            elif 'is' in line:
                key, val = line.split('is')
                facts[key.strip()] = val.strip()
            else:
                facts[line.strip()] = True
    return facts, target

In [2]:
def parse_condition(cond):
    cond = cond.strip()
    if '>=' in cond:
        key, val = cond.split('>=')
        return (key.strip(), '>=', float(val.strip()))
    elif '<=' in cond:
        key, val = cond.split('<=')
        return (key.strip(), '<=', float(val.strip()))
    if '>' in cond:
        key, val = cond.split('>')
        return (key.strip(), '>', float(val.strip()))
    elif '<' in cond:
        key, val = cond.split('<')
        return (key.strip(), '<', float(val.strip()))
    elif '=' in cond:
        key, val = cond.split('=')
        try:
            return (key.strip(), '=', int(val.strip()))
        except:
            return (key.strip(), '=', val.strip())
    elif 'is' in cond:
        key, val = cond.split('is')
        return (key.strip(), 'is', val.strip())
    else:
        return (cond.strip(), '==', True)


In [3]:
def parse_rules(file_path):
    rules = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or 'IF' not in line or 'THEN' not in line:
                continue
            cond_part, conclusion = line.split('THEN')
            cond_part = cond_part.replace('IF', '').strip()
            conclusion = conclusion.strip()

            and_conditions = [c.strip() for c in re.split(r'\bAND\b', cond_part)]
            condition_list = []
            for cond in and_conditions:
                if 'OR' in cond:
                    or_parts = [c.strip() for c in cond.split('OR')]
                    or_group = [parse_condition(c) for c in or_parts]
                    condition_list.append(('OR', or_group))
                else:
                    condition_list.append(parse_condition(cond))
            conclusion_parsed = parse_condition(conclusion)
            rules.append((condition_list, conclusion_parsed))
    return rules


In [4]:
def evaluate_condition(facts, cond):
    key, op, val = cond
    if key not in facts:
        return False
    try:
        if op == '=' or op == 'is':
            return facts[key] == val
        elif op == '==':
            return bool(facts[key])
        elif op == '>':
            return float(facts[key]) > float(val)
        elif op == '<':
            return float(facts[key]) < float(val)
        elif op == '>=':
            return float(facts[key]) >= float(val)
        elif op == '<=':
            return float(facts[key]) <= float(val)
    except:
        return False
    return False


In [5]:
def evaluate_conditions(facts, conditions):
    for cond in conditions:
        if isinstance(cond, tuple) and cond[0] == 'OR':
            if not any(evaluate_condition(facts, c) for c in cond[1]):
                return False
        else:
            if not evaluate_condition(facts, cond):
                return False
    return True


In [6]:
def forward_chaining(facts, rules, target=None):
    changed = True
    while changed:
        changed = False
        print("Current facts:", facts)
        for conditions, conclusion in rules:
            key, op, val = conclusion
            if key in facts and facts[key] == val:
                continue
            if evaluate_conditions(facts, conditions):
                facts[key] = val
                changed = True
                if target and key == target[0] and facts[key] == target[1]:
                    print(f"\nTarget reached: {key} ")
                    return facts
    return facts


In [7]:
def backward_chaining(facts, rules, target, visited=None):
    if visited is None:
        visited = set()

    key, val = target

    if key in facts and facts[key] == val:
        return True

    if key in visited:
        return False  

    visited.add(key)

    for conditions, conclusion in rules:
        key_con, op_con, value_con = conclusion

        if key_con == key and value_con == val:
            condition_met = True

            for cond in conditions:
                if isinstance(cond, tuple) and cond[0] == 'OR':
                    or_group = cond[1]
                    if not any(backward_chaining(facts, rules, (c[0], c[2]), visited) for c in or_group):
                        condition_met = False
                        break
                else:
                    if not backward_chaining(facts, rules, (cond[0], cond[2]), visited):
                        condition_met = False
                        break

            if condition_met:
                facts[key] = val
                return True

    return False  
  

In [8]:
def backward_chaining2(facts, rules, target, visited=None, depth=0):
    if visited is None:
        visited = set()

    indent = "  " * depth  # Indentation for visualizing recursion
    key, val = target

    print(f"{indent}Trying to prove: {key} = {val}")

    # Already known
    if key in facts and facts[key] == val:
        print(f"{indent}Already known: {key} = {val}")
        return True

    # Avoid loops
    if key in visited:
        print(f"{indent}Already visited: {key}, skipping to avoid loop.")
        return False

    visited.add(key)

    for conditions, conclusion in rules:
        key_con, op_con, value_con = conclusion

        if key_con == key and value_con == val:
            print(f"{indent}Checking rule: IF {conditions} THEN {conclusion}")
            condition_met = True

            for cond in conditions:
                if isinstance(cond, tuple) and cond[0] == 'OR':
                    or_group = cond[1]
                    print(f"{indent}Trying OR group: {or_group}")
                    if not any(backward_chaining2(facts, rules, (c[0], c[2]), visited, depth + 1) for c in or_group):
                        print(f"{indent}OR group failed.")
                        condition_met = False
                        break
                else:
                    print(f"{indent}Trying condition: {cond}")
                    if not backward_chaining2(facts, rules, (cond[0], cond[2]), visited, depth + 1):
                        print(f"{indent}Condition {cond} failed.")
                        condition_met = False
                        break

            if condition_met:
                facts[key] = val
                print(f"{indent}Rule succeeded, inferred: {key} = {val}")
                return True

    print(f"{indent}No rule could infer: {key} = {val}")
    return False



In [9]:
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [10]:
facts_back , target_back = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules_back = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [11]:
print("\n--- Forward Chaining ---")
final_facts = forward_chaining(facts, rules, target)

print("\nFinal inferred facts:")
for fact in final_facts:
    print(f"{fact} = {final_facts[fact]}")


--- Forward Chaining ---
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

Target reached: citrus_fruit 

Final inferred facts:
seeds = 0
diameter = 7
skin_smell = True
color = orange
perfumed = True
size = medium
fruit = orange
citrus_fruit = True


In [12]:
print("\n--- Backward Chaining ---")
if backward_chaining2(facts_back, rules_back, target_back):
    print(f"Target {target_back[0]} was proven.")
else:
    print(f"Target {target_back[0]} could not be proven.")


--- Backward Chaining ---
Trying to prove: citrus_fruit = True
Checking rule: IF [('OR', [('fruit', 'is', 'lemon'), ('fruit', 'is', 'orange'), ('fruit', 'is', 'pomelo'), ('fruit', 'is', 'grapefruit')])] THEN ('citrus_fruit', '==', True)
Trying OR group: [('fruit', 'is', 'lemon'), ('fruit', 'is', 'orange'), ('fruit', 'is', 'pomelo'), ('fruit', 'is', 'grapefruit')]
  Trying to prove: fruit = lemon
  Checking rule: IF [('size', 'is', 'medium'), ('color', 'is', 'yellow'), ('perfumed', '==', True)] THEN ('fruit', 'is', 'lemon')
  Trying condition: ('size', 'is', 'medium')
    Trying to prove: size = medium
    Checking rule: IF [('diameter', '>', 2.0), ('diameter', '<', 10.0)] THEN ('size', 'is', 'medium')
    Trying condition: ('diameter', '>', 2.0)
      Trying to prove: diameter = 2.0
      No rule could infer: diameter = 2.0
    Condition ('diameter', '>', 2.0) failed.
    No rule could infer: size = medium
  Condition ('size', 'is', 'medium') failed.
  No rule could infer: fruit = lem